In [ ]:
!pip install lazypredict

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer as s
from sklearn.impute import KNNImputer as knn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
data = pd.read_csv(r'train.csv')
data_pred=pd.read_csv(r'test.csv')

In [ ]:
class PreProcess():
    #Auto Run Upon Initiation .
    def __init__(self,data):
        self.data=data
        self.run()
    # PreProcessing  Schedules
    def run(self):
        self.ClearNull(threshold=0.5)
        l=self.get_all_Null(dtype='float64')
        self.knn_impute(2,l)
        #self.outlier_remove()
        self.data=self.data.dropna()
        self.one_hot_encoding()
        #self.Label_Encoding()
        self.StdScale()
        #self.outlier_remove('SalePrice')
        #self.drop_correlation()

        #self.drop_vif(thresh=4.5)

    # Remove Correlation
    def drop_correlation(self):
        k=Utils_Suite(self.data).compute_correlation(0.3)
        f=pd.DataFrame(k)
        m=list(f[(f['SalePrice']<0.1) & (f['SalePrice']>-0.1)].index)
        self.data=self.data.drop(columns=m)

    def ClearNull(self,threshold):
        x=self.data.isna().sum()>0

        for i in  list(x.index):
            thresh=self.data[i].isna().sum()/len(self.data)
            if(x[i]==True and thresh>threshold):
                print(i,self.data[i].isna().sum())
                self.data=self.data.drop(i,axis=1)


    def knn_impute(self,n_neighbors,col_list):
        imputer=knn(n_neighbors=n_neighbors)
        for i in col_list:
            self.data[i]=imputer.fit_transform(self.data[[i]])[0][0]

    def arbitrary_remove(self):
        self.data=data.drop(columns=['LotFrontage','MasVnrArea','GarageYrBlt'])
        self.data=self.data.drop('Id',axis=1)

    def get_all_Null(self,dtype=""):
        x=self.data.isna().sum()>0
        l=[]
        for i in  list(x.index):
            thresh=self.data[i].isna().sum()/len(self.data)
            if(x[i]==True and (data[i].dtypes==dtype) ):
                print(i,data[i].isna().sum())
                l+=[i]
        return l
    # Drop Outlier Rows
    def outlier_remove(self,col):

        q1=self.data[col].quantile(0.25)
        q3=self.data[col].quantile(0.75)
        iqr=q3-q1
        l_whis=q1-1.5*iqr
        u_whis=q3+1.5*iqr
        self.data= self.data[(self.data[col]>=l_whis)& (self.data[col]<=u_whis)]

    #Depricated  ....(DONT USE IT - Diminishes the data set )
    def outlier_remove_deprecated(self):
        for col in self.data.columns:
            if self.data[col].dtypes!='object':

                q1=self.data[col].quantile(0.25)
                q3=self.data[col].quantile(0.75)
                iqr=q3-q1
                l_whis=q1-1.5*iqr
                u_whis=q3+1.5*iqr
                self.data= self.data[(self.data[col]>=l_whis)& (self.data[col]<=u_whis)]
        return self.data
    # One hot Encoding using get_dummies
    def one_hot_encoding(self):
        z=(self.data.dtypes=='object')
        k=pd.DataFrame(z)
        obj_list=list(k[k[0]==True].index)
        print(obj_list)
        for i in obj_list:
            dummy=pd.get_dummies(self.data[i],prefix=i,drop_first=True)
            #print(dummy)
            self.data=self.data.drop(i,axis=1)
            self.data=self.data.join(dummy)
            #self.data=pd.concat([self.data,dummy],axis=1)
    # standardize data
    def Label_Encoding(self):
        label_encoder = LabelEncoder()
        x=pd.DataFrame(self.data.dtypes)
        ll=list(x[x[0]=="object"].index)
        for i in ll:
            self.data[i]= label_encoder.fit_transform(self.data[i])
    def StdScale(self):
        for i in self.data.columns:
            if self.data[i].dtypes!='object' and i!='SalePrice':
                scale = StandardScaler().fit(self.data[[i]])

                self.data[i] = scale.transform(self.data[[i]])


    ## DANGER ZONE Col Spare NEEDED To Keep y_pred. RAM HOGGING FUNCTION .
    #Use Wisely! Plus Parallize the operation for better efficacy? Maybe???

    def drop_vif(self,thresh=5,col_Spare=['SalePrice','intercept']):


        vif=Utils_Suite(self.data).compute_vif()
        z1=vif[vif["vif"]>thresh]
        z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
        while True:
            try:
                col=z1.iloc[0,0]
                if z1.empty:
                    break
                if col in col_Spare:
                    z1=z1.iloc[1:]
                    continue
                self.data=self.data.drop(col,axis=1)
                vif=Utils_Suite(self.data).compute_vif()
                z1=vif[vif["vif"]>thresh]
                z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
            except IndexError:
                break

    # Wrapper Function to dump data to a variable
    def write_df(self):
        return self.data

In [ ]:
def one_hot_encoding(data):
        z=(data.dtypes=='object')
        k=pd.DataFrame(z)
        obj_list=list(k[k[0]==True].index)
        print(obj_list)
        for i in obj_list:
            dummy=pd.get_dummies(data[i],prefix=i,drop_first=True)
            #print(dummy)
            data=data.drop(i,axis=1)
            data=data.join(dummy)
        return data

In [ ]:
def Label_Encoding(data):
        label_encoder = LabelEncoder()
        x=pd.DataFrame(data.dtypes)
        ll=list(x[x[0]=="object"].index)
        for i in ll:
            data[i]= label_encoder.fit_transform(data[i])
        return (data,label_encoder)

In [ ]:
class Utils_Suite():
    def __init__(self,data):
        self.data=data
    def compute_correlation(self,threshold=0.3):
        matrix=self.data.corr(numeric_only=True)
        x=matrix[(matrix["SalePrice"]<threshold)&(matrix["SalePrice"]>-threshold)]["SalePrice"]
        return x
    def compute_mutual_information(self,thresh=0.1):
        enc = OrdinalEncoder()
        df_encoded = enc.fit_transform(self.data)
        mi_scores = mutual_info_regression(df_encoded, self.data['SalePrice'])
        mi_scores_df = pd.DataFrame(mi_scores, index=self.data.columns, columns=['Score'])
        return mi_scores_df[mi_scores_df['Score']<thresh]
    def compute_vif(self):
        x=self.data.iloc[:,:-1]
        y=self.data.iloc[:,-1]
        x=pd.DataFrame(x)

        x['intercept']=1
        vif=pd.DataFrame()
        vif['variable']=x.columns
        vif['vif']=[variance_inflation_factor(x.values,i)for i in range(x.shape[1])]
        return vif

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
def ClearNull(data,threshold):

  x=data.isna().sum()>0

  for i in  list(x.index):
    thresh=data[i].isna().sum()/len(data)
    if(x[i]==True and thresh>threshold):
      print(i,data[i].isna().sum())
      data=data.drop(i,axis=1)
  return data

In [ ]:
def StdScale(data):
        for i in data.columns:
            if data[i].dtypes!='object' and i!='SalePrice':
                scale = StandardScaler().fit(data[[i]])

                data[i] = scale.transform(data[[i]])
        return data

In [ ]:
def get_all_Null(data,dtype=""):
        x=data.isna().sum()>0
        l=[]
        for i in  list(x.index):
            thresh=data[i].isna().sum()/len(data)
            if(x[i]==True and (data[i].dtypes==dtype) ):
                print(i,data[i].isna().sum())
                l+=[i]
        return l

In [ ]:
data=ClearNull(data,threshold=0.4)

NameError: name 'data' is not defined

In [ ]:
data1=StdScale(data1)

In [ ]:
data=one_hot_encoding(data)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


In [ ]:
data

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.00,8450,7,5,2003,2003,196.00,706,...,False,False,False,False,True,False,False,False,True,False
1,2,20,80.00,9600,6,8,1976,1976,0.00,978,...,False,False,False,False,True,False,False,False,True,False
2,3,60,68.00,11250,7,5,2001,2002,162.00,486,...,False,False,False,False,True,False,False,False,True,False
3,4,70,60.00,9550,7,5,1915,1970,0.00,216,...,False,False,False,False,True,False,False,False,False,False
4,5,60,84.00,14260,8,5,2000,2000,350.00,655,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.00,7917,6,5,1999,2000,0.00,0,...,False,False,False,False,True,False,False,False,True,False
1456,1457,20,85.00,13175,6,6,1978,1988,119.00,790,...,False,False,False,False,True,False,False,False,True,False
1457,1458,70,66.00,9042,7,9,1941,2006,0.00,275,...,False,False,False,False,True,False,False,False,True,False
1458,1459,20,68.00,9717,5,6,1950,1996,0.00,49,...,False,False,False,False,True,False,False,False,True,False


In [ ]:
le=Label_Encoding(data)

In [ ]:
le_o=le[-1]

In [ ]:
data=le[0]

In [ ]:
data

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,-1.73,0.07,-0.05,-0.21,0.06,0.75,0.31,-0.03,0.60,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,-1.60,0.14,0.31,0.21,208500
1,-1.73,-0.87,-0.05,-0.09,0.06,0.75,0.31,-0.03,-0.63,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,-0.49,-0.61,0.31,0.21,181500
2,-1.73,0.07,-0.05,0.07,0.06,-1.38,0.31,-0.03,0.60,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,0.99,0.14,0.31,0.21,223500
3,-1.72,0.31,-0.05,-0.10,0.06,-1.38,0.31,-0.03,-1.86,-0.23,...,4.09,-0.12,-0.27,-0.07,-0.09,-1.60,-1.37,0.31,-3.43,140000
4,-1.72,0.07,-0.05,0.38,0.06,-1.38,0.31,-0.03,-0.63,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,2.10,0.14,0.31,0.21,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.72,0.07,-0.05,-0.26,0.06,0.75,0.31,-0.03,0.60,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,0.62,-0.61,0.31,0.21,175000
1456,1.72,-0.87,-0.05,0.27,0.06,0.75,0.31,-0.03,0.60,-0.23,...,-0.36,-0.12,-0.27,-0.07,-0.09,-1.60,1.65,0.31,0.21,210000
1457,1.73,0.31,-0.05,-0.15,0.06,0.75,0.31,-0.03,0.60,-0.23,...,-0.36,-0.12,-0.27,-0.07,4.95,-0.49,1.65,0.31,0.21,266500
1458,1.73,-0.87,-0.05,-0.08,0.06,0.75,0.31,-0.03,0.60,-0.23,...,1.47,-0.12,-0.27,-0.07,-0.09,-0.86,1.65,0.31,0.21,142125


In [ ]:
def drop_correlation(data):
        k=Utils_Suite(data).compute_correlation(0.3)
        f=pd.DataFrame(k)
        m=list(f[(f['SalePrice']<0.1) & (f['SalePrice']>-0.1)].index)
        data=data.drop(columns=m)
        print(m)
        return data

In [ ]:
data1=drop_correlation(data1)

KeyError: 'SalePrice'

In [ ]:
def drop_vif(data,thresh=5,col_Spare=['SalePrice','intercept']):


        vif=Utils_Suite(data).compute_vif()
        z1=vif[vif["vif"]>thresh]
        z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
        while True:
            try:
                col=z1.iloc[0,0]
                if z1.empty:
                    break
                if col in col_Spare:
                    z1=z1.iloc[1:]
                    continue
                data=data.drop(col,axis=1)
                vif=Utils_Suite(data).compute_vif()
                z1=vif[vif["vif"]>thresh]
                z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
            except IndexError:
                break
        return data

In [ ]:
data.dtypes

MSZoning           int64
LotFrontage      float64
LotArea          float64
LotShape           int64
Neighborhood       int64
HouseStyle         int64
OverallQual      float64
YearBuilt        float64
YearRemodAdd     float64
RoofStyle          int64
RoofMatl           int64
Exterior1st        int64
Exterior2nd        int64
MasVnrArea       float64
ExterQual          int64
ExterCond          int64
Foundation         int64
BsmtQual           int64
BsmtExposure       int64
BsmtFinType1       int64
BsmtFinSF1       float64
BsmtUnfSF        float64
TotalBsmtSF      float64
HeatingQC          int64
CentralAir         int64
Electrical         int64
1stFlrSF         float64
2ndFlrSF         float64
GrLivArea        float64
BsmtFullBath     float64
FullBath         float64
HalfBath         float64
BedroomAbvGr     float64
KitchenAbvGr     float64
KitchenQual        int64
TotRmsAbvGrd     float64
Functional         int64
Fireplaces       float64
GarageType         int64
GarageYrBlt      float64


In [ ]:
get_all_Null(data,dtype="bool")

[]

In [ ]:
data

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,GarageFinish_Unf,GarageQual_Fa,GarageQual_TA,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_New,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,8450,7,2003,2003,706,150,856,856,854,1710,...,False,False,True,False,True,True,False,True,True,False
1,9600,6,1976,1976,978,284,1262,1262,0,1262,...,False,False,True,False,True,True,False,True,True,False
2,11250,7,2001,2002,486,434,920,920,866,1786,...,False,False,True,False,True,True,False,True,True,False
3,9550,7,1915,1970,216,540,756,961,756,1717,...,True,False,True,False,True,True,False,True,False,False
4,14260,8,2000,2000,655,490,1145,1145,1053,2198,...,False,False,True,False,True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,1999,2000,0,953,953,953,694,1647,...,False,False,True,False,True,True,False,True,True,False
1456,13175,6,1978,1988,790,589,1542,2073,0,2073,...,True,False,True,False,True,True,False,True,True,False
1457,9042,7,1941,2006,275,877,1152,1188,1152,2340,...,False,False,True,False,True,True,False,True,True,False
1458,9717,5,1950,1996,49,0,1078,1078,0,1078,...,True,False,True,False,True,True,False,True,True,False


In [ ]:
data=data.drop(columns=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'],axis=1)

In [ ]:
data1=dt.iloc[:,:-1]

In [ ]:
data1

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,GarageFinish_Unf,GarageQual_Fa,GarageQual_TA,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_New,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,8450,7,2003,2003,706,150,856,856,854,1710,...,False,False,True,False,True,True,False,True,True,False
1,9600,6,1976,1976,978,284,1262,1262,0,1262,...,False,False,True,False,True,True,False,True,True,False
2,11250,7,2001,2002,486,434,920,920,866,1786,...,False,False,True,False,True,True,False,True,True,False
3,9550,7,1915,1970,216,540,756,961,756,1717,...,True,False,True,False,True,True,False,True,False,False
4,14260,8,2000,2000,655,490,1145,1145,1053,2198,...,False,False,True,False,True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,1999,2000,0,953,953,953,694,1647,...,False,False,True,False,True,True,False,True,True,False
1456,13175,6,1978,1988,790,589,1542,2073,0,2073,...,True,False,True,False,True,True,False,True,True,False
1457,9042,7,1941,2006,275,877,1152,1188,1152,2340,...,False,False,True,False,True,True,False,True,True,False
1458,9717,5,1950,1996,49,0,1078,1078,0,1078,...,True,False,True,False,True,True,False,True,True,False


In [ ]:
data1=drop_vif(data1,thresh=3)

In [ ]:
data1

,LotArea,YearRemodAdd,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,...,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_RFn,GarageQual_Fa,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,-0.21,0.88,-0.94,1.11,0.79,1.23,0.16,-0.21,-0.95,0.35,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
1,-0.09,-0.43,-0.64,-0.82,0.79,-0.76,0.16,-0.21,0.60,-0.06,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
2,0.07,0.83,-0.30,1.11,0.79,1.23,0.16,-0.21,0.60,0.63,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
3,-0.10,-0.72,-0.06,1.11,-1.03,-0.76,0.16,-0.21,0.60,0.79,...,-0.25,1.67,-0.64,-0.18,-0.16,0.32,0.30,0.39,-2.14,-0.31
4,0.38,0.73,-0.17,1.11,0.79,1.23,1.39,-0.21,0.60,1.70,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.26,0.73,0.87,-0.82,0.79,1.23,0.16,-0.21,0.60,-0.06,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
1456,0.27,0.15,0.05,1.11,0.79,-0.76,0.16,-0.21,2.15,0.13,...,-0.25,-0.60,-0.64,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
1457,-0.15,1.02,0.70,-0.82,0.79,-0.76,1.39,-0.21,2.15,-1.03,...,-0.25,-0.60,1.57,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31
1458,-0.08,0.54,-1.28,1.11,-1.03,-0.76,-1.06,-0.21,-0.95,-1.09,...,-0.25,-0.60,-0.64,-0.18,-0.16,0.32,0.30,0.39,0.47,-0.31


In [ ]:
data.dtypes

LotArea                  int64
OverallQual              int64
YearBuilt                int64
YearRemodAdd             int64
BsmtFinSF1               int64
                         ...  
PavedDrive_Y              bool
SaleType_New              bool
SaleType_WD               bool
SaleCondition_Normal      bool
SaleCondition_Partial     bool
Length: 101, dtype: object

In [ ]:
dt=k.data

In [ ]:
dt=data

In [ ]:
m=list(dt.columns)

In [ ]:
m.remove('SalePrice')
m.append('SalePrice')

In [ ]:
dt=dt[m]

In [ ]:
dt.iloc[:,-1]

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x=data1.values
y=dt.iloc[:,-1].values

In [ ]:
x

array([[-0.20714171,  0.65147924,  0.87866809, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.09188637, -0.07183611, -0.42957697, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [ 0.07347998,  0.65147924,  0.83021457, ...,  0.39029263,
         0.4676514 , -0.30599503],
       ...,
       [-0.14781027,  0.65147924,  1.02402865, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.08016039, -0.79515147,  0.53949344, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.05811155, -0.79515147, -0.96256569, ...,  0.39029263,
         0.4676514 , -0.30599503]])

In [ ]:
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.32,random_state =123)


In [ ]:
X_test

array([[-0.10140747,  0.78176105,  0.71711206, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.09188637, -1.6893685 ,  1.19479498, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.71697118,  0.92712161,  0.92539087, ...,  0.39029263,
         0.4676514 , -0.30599503],
       ...,
       [ 2.96428424,  1.12093569,  0.70352866, ..., -2.5621801 ,
        -2.13834494,  3.26802693],
       [-0.09188637,  0.4425864 ,  0.30508225, ...,  0.39029263,
         0.4676514 , -0.30599503],
       [-0.20714171,  1.16938921, -0.777062  , ...,  0.39029263,
         0.4676514 , -0.30599503]])

In [ ]:
from lazypredict.Supervised import LazyRegressor


In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)


In [ ]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)


 74%|███████▍  | 31/42 [00:06<00:02,  4.94it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:09<00:00,  4.22it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000610 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1285
[LightGBM] [Info] Number of data points in the train set: 992, number of used features: 67
[LightGBM] [Info] Start training from score 179635.149194


In [ ]:
print(models)

                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
PoissonRegressor                             0.81       0.84  31171.77   
GradientBoostingRegressor                    0.80       0.83  32173.34   
HistGradientBoostingRegressor                0.78       0.81  33717.80   
ExtraTreesRegressor                          0.78       0.81  33912.72   
RandomForestRegressor                        0.77       0.81  34441.40   
LGBMRegressor                                0.76       0.80  34718.84   
SGDRegressor                                 0.76       0.80  34984.42   
BayesianRidge                                0.76       0.80  35179.23   
LassoCV                                      0.76       0.80  35269.26   
LassoLarsIC                                  0.76       0.80  35269.98   
LassoLarsCV                                  0.76       0.80  35272.68   
RidgeCV                               

{'alpha': 4.75, 'fit_intercept': True, 'max_iter': 100, 'solver': 'lbfgs'}


In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor(alpha=4.75,fit_intercept=True,max_iter=100,solver='lbfgs')
clf.fit(X_train, y_train)

PoissonRegressor(alpha=4.75)

In [ ]:
reg = ExtraTreesRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
reg.score(X_test,y_test)

0.8056777830735948

In [ ]:
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [ ]:
y_pred=reg.predict(X_test)

In [ ]:
y_pred

array([209858.  , 135365.04, 135657.3 , 213663.4 , 148863.  , 254513.66,
       297185.6 , 160408.05, 157145.53,  89835.  , 159969.5 , 217599.55,
       212355.37, 115718.35, 226132.1 , 197401.5 , 133555.5 , 366419.45,
       209030.21, 164899.59, 147110.36, 203249.34, 109226.32, 136063.05,
       223124.3 , 156320.25, 215345.  , 206766.21, 127320.55, 130922.15,
       148903.5 , 230915.95, 164589.04, 226644.94, 289754.79, 193818.  ,
       189418.65, 275881.59, 162170.09,  91071.86, 128628.  , 211579.77,
       198417.  , 163040.  , 210982.58, 116894.  , 323160.12, 290677.89,
       245131.96, 126871.59, 253898.05, 215982.2 , 144745.33, 154781.  ,
        85396.04, 157911.5 , 131995.  , 141516.5 , 237770.66, 391681.79,
       147504.55, 293613.  , 143305.  , 168701.  , 202538.35, 128307.  ,
       129842.5 , 247859.2 , 236145.46, 131200.  , 143347.43, 200714.28,
       166446.01, 254076.35, 157065.  , 140162.25, 188867.4 , 128302.51,
       224321.47, 137644.75, 315468.72, 255122.93, 

In [ ]:
scale = StandardScaler()
original_price_predictions = scale.inverse_transform(y_pred)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
y_test

array([222500,  87000, 130000, 274300, 140000, 259000, 309000, 127500,
       155000, 147000, 159000, 220000, 140000, 108000, 260000, 207500,
       115000, 325624, 212000, 167900, 151000, 162900, 109500, 143000,
       230000, 145000, 208900, 181000, 129900, 107000, 149350, 182000,
       143000, 198900, 367294, 190000, 173000, 475000, 176500, 101000,
       144000, 174000, 215000, 165000, 215000, 135750, 311872, 248000,
       239900, 116000, 200000, 196000, 140000, 143000,  89500, 153000,
       144000, 168000, 228000, 360000, 108000, 345000, 122000, 167000,
       244000, 140000, 120000, 279500, 232600, 140000, 135000, 129000,
       171900, 250000, 169500, 140000, 183200, 140000, 210000, 133000,
       313000, 284000,  85500, 125000, 168000, 140000,  98000, 129000,
        52500, 302000, 127500, 181000, 178000, 163000, 153337, 119500,
       244400, 109500, 219210, 133500, 104900, 582933, 342643, 148000,
       207500, 115000, 129000, 190000, 177500, 318000, 280000, 242000,
      

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(np.log(y_test),np.log(y_pred)))

0.15815736107650438

In [ ]:
y_test[1],y_pred[1]

(-0.30599503068105227, -0.305995030681053)

In [ ]:
y_pred

array([-0.3596155 , -0.3596155 ,  2.78074777, -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 ,  2.78074777, -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.32821187, -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 ,  2.78074777, -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 ,  2.78074777,  2.78074777,
        2.78074777, -0.3596155 ,  2.78074777, -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 ,
       -0.3596155 , -0.3596155 , -0.3596155 , -0.3596155 , -0.35

In [ ]:
data1

,LotArea,YearRemodAdd,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,...,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_RFn,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,-0.21,0.88,-0.94,1.11,0.79,1.23,0.16,-0.21,-0.95,0.35,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
1,-0.09,-0.43,-0.64,-0.82,0.79,-0.76,0.16,-0.21,0.60,-0.06,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
2,0.07,0.83,-0.30,1.11,0.79,1.23,0.16,-0.21,0.60,0.63,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
3,-0.10,-0.72,-0.06,1.11,-1.03,-0.76,0.16,-0.21,0.60,0.79,...,0.27,-0.25,1.67,-0.64,-0.16,0.32,0.30,0.39,-2.14,-0.31
4,0.38,0.73,-0.17,1.11,0.79,1.23,1.39,-0.21,0.60,1.70,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.26,0.73,0.87,-0.82,0.79,1.23,0.16,-0.21,0.60,-0.06,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
1456,0.27,0.15,0.05,1.11,0.79,-0.76,0.16,-0.21,2.15,0.13,...,-3.69,-0.25,-0.60,-0.64,-0.16,0.32,0.30,0.39,0.47,-0.31
1457,-0.15,1.02,0.70,-0.82,0.79,-0.76,1.39,-0.21,2.15,-1.03,...,0.27,-0.25,-0.60,1.57,-0.16,0.32,0.30,0.39,0.47,-0.31
1458,-0.08,0.54,-1.28,1.11,-1.03,-0.76,-1.06,-0.21,-0.95,-1.09,...,0.27,-0.25,-0.60,-0.64,-0.16,0.32,0.30,0.39,0.47,-0.31


In [ ]:
col=data1.columns

In [ ]:
col=list(col)

In [ ]:
data_pred=one_hot_encoding(data_pred)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [ ]:
data['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
col.remove('SalePrice')

In [ ]:
col.remove('GarageQual_Fa')
col.remove('RoofMatl_CompShg')

In [ ]:
data_pred=data_pred[col]

In [ ]:
data1['RoofMatl_CompShg']

0      0.13
1      0.13
2      0.13
3      0.13
4      0.13
       ... 
1455   0.13
1456   0.13
1457   0.13
1458   0.13
1459   0.13
Name: RoofMatl_CompShg, Length: 1460, dtype: float64

In [ ]:
p11=list(data1.columns)

In [ ]:
p12=list(data_pred.columns)

In [ ]:
data_pred

,LotArea,YearRemodAdd,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,...,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_RFn,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,11622,1961,270.00,0.00,1,0,2,1,0,730.00,...,True,False,False,False,False,True,True,True,True,False
1,14267,1958,406.00,0.00,1,1,3,1,0,312.00,...,True,False,False,False,False,True,True,True,True,False
2,13830,1998,137.00,0.00,2,1,3,1,1,482.00,...,True,False,False,False,False,True,True,True,True,False
3,9978,1998,324.00,0.00,2,1,3,1,1,470.00,...,True,False,False,False,False,True,True,True,True,False
4,5005,1992,1017.00,0.00,2,0,2,1,0,506.00,...,True,False,False,True,False,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,1970,546.00,0.00,1,1,3,1,0,0.00,...,True,False,False,False,False,False,True,True,True,False
1455,1894,1970,294.00,0.00,1,1,3,1,0,286.00,...,True,False,False,False,False,True,True,True,False,False
1456,20000,1996,0.00,1.00,1,0,4,1,1,576.00,...,True,False,True,False,False,True,True,True,False,False
1457,10441,1992,575.00,0.00,1,0,3,1,0,0.00,...,True,False,False,False,False,False,True,True,True,False


In [ ]:
drop=list(set(p11)-set(p12))

In [ ]:
drop

['RoofMatl_CompShg', 'GarageQual_Fa']

In [ ]:
data_pred['FireplaceQu_Po ']

KeyError: 'FireplaceQu_Po'

In [ ]:
dt[]

In [ ]:
drop

['FireplaceQu_Po', 'FireplaceQu_Gd', 'SalePrice', 'FireplaceQu_Fa']

In [ ]:
data_pred

,LotArea,YearRemodAdd,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,...,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_RFn,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,11622,1961,270.00,0.00,1,0,2,1,0,730.00,...,True,False,False,False,False,True,True,True,True,False
1,14267,1958,406.00,0.00,1,1,3,1,0,312.00,...,True,False,False,False,False,True,True,True,True,False
2,13830,1998,137.00,0.00,2,1,3,1,1,482.00,...,True,False,False,False,False,True,True,True,True,False
3,9978,1998,324.00,0.00,2,1,3,1,1,470.00,...,True,False,False,False,False,True,True,True,True,False
4,5005,1992,1017.00,0.00,2,0,2,1,0,506.00,...,True,False,False,True,False,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,1970,546.00,0.00,1,1,3,1,0,0.00,...,True,False,False,False,False,False,True,True,True,False
1455,1894,1970,294.00,0.00,1,1,3,1,0,286.00,...,True,False,False,False,False,True,True,True,False,False
1456,20000,1996,0.00,1.00,1,0,4,1,1,576.00,...,True,False,True,False,False,True,True,True,False,False
1457,10441,1992,575.00,0.00,1,0,3,1,0,0.00,...,True,False,False,False,False,False,True,True,True,False


In [ ]:
get_all_Null(data_pred,dtype="float64")

BsmtUnfSF 1
BsmtFullBath 2
GarageArea 1


['BsmtUnfSF', 'BsmtFullBath', 'GarageArea']

In [ ]:
def knn_impute(data,n_neighbors,col_list):
        imputer=knn(n_neighbors=n_neighbors)
        for i in col_list:
            data[i]=imputer.fit_transform(data[[i]])[0][0]
        return data

In [ ]:
data_pred=knn_impute(data_pred,3,['BsmtUnfSF', 'BsmtFullBath', 'GarageArea'])

In [ ]:
data_pred

,LotArea,YearRemodAdd,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,...,Functional_Typ,GarageType_BuiltIn,GarageType_Detchd,GarageFinish_RFn,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,11622,1961,270.00,0.00,1,0,2,1,0,730.00,...,True,False,False,False,False,True,True,True,True,False
1,14267,1958,270.00,0.00,1,1,3,1,0,730.00,...,True,False,False,False,False,True,True,True,True,False
2,13830,1998,270.00,0.00,2,1,3,1,1,730.00,...,True,False,False,False,False,True,True,True,True,False
3,9978,1998,270.00,0.00,2,1,3,1,1,730.00,...,True,False,False,False,False,True,True,True,True,False
4,5005,1992,270.00,0.00,2,0,2,1,0,730.00,...,True,False,False,True,False,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1936,1970,270.00,0.00,1,1,3,1,0,730.00,...,True,False,False,False,False,False,True,True,True,False
1455,1894,1970,270.00,0.00,1,1,3,1,0,730.00,...,True,False,False,False,False,True,True,True,False,False
1456,20000,1996,270.00,0.00,1,0,4,1,1,730.00,...,True,False,True,False,False,True,True,True,False,False
1457,10441,1992,270.00,0.00,1,0,3,1,0,730.00,...,True,False,False,False,False,False,True,True,True,False


In [ ]:
data_pred=StdScale(data_pred)

In [ ]:
d12=drop_vif(data_pred)

In [ ]:
d12

,LotArea,OverallQual,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,...,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_New,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,0.36,-0.75,-1.07,0.00,0.00,0.00,-1.03,-0.75,-1.03,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
1,0.90,-0.05,-1.21,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
2,0.81,-0.75,0.68,0.00,0.00,0.00,0.77,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
3,0.03,-0.05,0.68,0.00,0.00,0.00,0.77,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
4,-0.97,1.34,0.39,0.00,0.00,0.00,0.77,-0.75,-1.03,-0.20,...,-0.61,1.66,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,-1.59,-1.45,-0.65,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,-3.18,0.35,-0.30,0.40,0.46,-0.30
1455,-1.60,-1.45,-0.65,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,-2.17,-0.30
1456,2.06,-0.75,0.58,0.00,0.00,0.00,-1.03,-0.75,1.38,-0.20,...,1.65,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,-2.17,-0.30
1457,0.13,-0.75,0.39,0.00,0.00,0.00,-1.03,-0.75,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,-3.18,0.35,-0.30,0.40,0.46,-0.30


In [ ]:
data_pred.values

array([[ 0.36392912, -0.75110125, -1.07288463, ...,  0.39972168,
         0.46021084, -0.29936452],
       [ 0.89786065, -0.05487716, -1.21490841, ...,  0.39972168,
         0.46021084, -0.29936452],
       [ 0.80964587, -0.75110125,  0.6787419 , ...,  0.39972168,
         0.46021084, -0.29936452],
       ...,
       [ 2.05514965, -0.75110125,  0.58405938, ...,  0.39972168,
        -2.17291708, -0.29936452],
       [ 0.12552719, -0.75110125,  0.39469435, ...,  0.39972168,
         0.46021084, -0.29936452],
       [-0.03879049,  0.64134693,  0.48937687, ...,  0.39972168,
         0.46021084, -0.29936452]])

In [ ]:
y_pred_data=reg.predict(data_pred.values)

In [ ]:
data1=data1.drop(columns=['GarageQual_Fa', 'RoofMatl_CompShg'],axis=1)

In [ ]:
set(data.columns)-set(data_pred.columns)

{'GarageQual_Fa', 'RoofMatl_CompShg', 'SalePrice'}

In [ ]:
data_pred

,LotArea,OverallQual,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,...,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageCond_Fa,GarageCond_TA,PavedDrive_Y,SaleType_New,SaleType_WD,SaleCondition_Normal,SaleCondition_Partial
0,0.36,-0.75,-1.07,0.00,0.00,0.00,-1.03,-0.75,-1.03,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
1,0.90,-0.05,-1.21,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
2,0.81,-0.75,0.68,0.00,0.00,0.00,0.77,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
3,0.03,-0.05,0.68,0.00,0.00,0.00,0.77,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
4,-0.97,1.34,0.39,0.00,0.00,0.00,0.77,-0.75,-1.03,-0.20,...,-0.61,1.66,-0.87,-0.17,0.31,0.35,-0.30,0.40,0.46,-0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,-1.59,-1.45,-0.65,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,-3.18,0.35,-0.30,0.40,0.46,-0.30
1455,-1.60,-1.45,-0.65,0.00,0.00,0.00,-1.03,1.24,0.18,-0.20,...,-0.61,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,-2.17,-0.30
1456,2.06,-0.75,0.58,0.00,0.00,0.00,-1.03,-0.75,1.38,-0.20,...,1.65,-0.60,1.16,-0.17,0.31,0.35,-0.30,0.40,-2.17,-0.30
1457,0.13,-0.75,0.39,0.00,0.00,0.00,-1.03,-0.75,0.18,-0.20,...,-0.61,-0.60,-0.87,-0.17,-3.18,0.35,-0.30,0.40,0.46,-0.30


In [ ]:
data_pred=p1.data

In [ ]:
reg.predict(p1.data)

ValueError: X has 169 features, but ExtraTreesRegressor is expecting 75 features as input.

In [ ]:
data_pred1=pd.read_csv(r'test.csv')

In [ ]:
y_pred_data=pd.Series(y_pred_data)

In [ ]:
ds=pd.concat([pd.Series(data_pred1['Id'].values),y_pred_data],axis=1)

In [ ]:
ds.columns=['Id','SalePrice']

In [ ]:
ds.to_csv('sample_submission_poisson_param_tuned.csv',index=False)


In [ ]:
pd.Series(data_pred1['Id'].values)

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Length: 1459, dtype: int64

In [ ]:
y_pred_data

0      130625.00
1      154869.16
2      197461.70
3      230752.28
4      262629.04
          ...   
1454   108093.00
1455    99150.71
1456   135138.00
1457   140146.50
1458   183303.50
Length: 1459, dtype: float64

In [ ]:
poisson_pred=clf.predict(data_pred.values)

In [ ]:
y_pred_data=pd.Series(poisson_pred)

hyper parameter tuning:
      alpha : 0.0 ->5
      fit_intercept :true/false
      solver{‘lbfgs’, ‘newton-cholesky’}
      max_iter:100 ->400
      

In [ ]:
param={
    'alpha':[i for i in np.arange(0.0,5.0,0.25)],
    'fit_intercept':[True,False],
    'solver':['lbfgs', 'newton-cholesky'],
    'max_iter':[i for i in range(100,401,100)]

    }

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clf1 = linear_model.PoissonRegressor()

In [ ]:
gr=GridSearchCV(clf1,param_grid=param,cv=5)
gr.fit(X_train,y_train)
gr.best_params_

{'alpha': 4.75, 'fit_intercept': True, 'max_iter': 100, 'solver': 'lbfgs'}